In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append('../')

In [3]:
from fangraphsdownload import updatefangraphs

In [10]:
#updatefangraphs(link='https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=atc',path='hitters')
updatefangraphs(link='https://www.fangraphs.com/projections.aspx?pos=all&stats=pit&type=atc&team=0&lg=all&players=0',path='pitchers')
#updatefangraphs(link='https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=0&season=2022&month=0&season1=2022&ind=0&team=0&rost=0&age=0&filter=&players=0',path='positions')
#updatefangraphs(link='https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=0&season=2021&month=0&season1=2021&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=',path='positionsold', debug=True)


In [12]:
def GetPosition (array):
    if isinstance(array,np.ndarray):
        if array.__contains__('C'):
            return 'C'
        if array.__contains__('2B'):
            return '2B'
        if array.__contains__('SS'):
            return 'SS'
        if array.__contains__('3B'):
            return '3B'
        if array.__contains__('1B'):
            return '1B'
        if array.__contains__('OF'):
            return 'OF'
        else:
            return 'Util'
    else: 
        return 'Util'

In [15]:
atcbat = pd.read_csv("hitters/FanGraphs Leaderboard.csv")
atcpit = pd.read_csv("pitchers/FanGraphs Leaderboard.csv")

In [16]:
hittersmean = atcbat
pitchersmean = atcpit

In [ ]:
hittersmean

,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,wRC+,-1.1,WAR,-1.2,ADP,-1.3,InterSD,InterSK,IntraSD,playerid
0,Juan Soto,WSN,153,661,519,162,30,3,36,112,...,174,NaN,7.6,NaN,3.9,NaN,2.47,0.74,0.98,20123
1,Mike Trout,LAA,136,587,475,134,25,3,37,99,...,164,NaN,6.6,NaN,12.9,NaN,3.23,-1.19,0.74,10155
2,José Ramírez,CLE,148,633,551,149,35,4,35,100,...,140,NaN,5.9,NaN,3.2,NaN,2.94,0.20,0.85,13510
3,Mookie Betts,LAD,147,657,565,158,34,3,31,112,...,138,NaN,5.8,NaN,15.5,NaN,3.44,-0.21,0.65,13611
4,Vladimir Guerrero Jr.,TOR,152,650,568,174,32,2,41,107,...,159,NaN,5.6,NaN,5.3,NaN,4.17,0.04,1.44,19611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,Scott Schebler,NaN,11,33,30,6,1,0,1,3,...,43,NaN,-0.2,NaN,999.0,NaN,5.95,2.30,1.81,12225
621,Alex Blandino,NaN,27,72,60,13,3,0,0,8,...,71,NaN,-0.2,NaN,999.0,NaN,6.76,0.59,1.67,16271
622,Josh Reddick,NaN,45,155,142,35,7,1,3,16,...,75,NaN,-0.2,NaN,999.0,NaN,0.78,3.25,1.41,3892
623,Albert Pujols,NaN,47,172,161,38,6,0,8,17,...,84,NaN,-0.4,NaN,598.2,NaN,3.13,0.09,1.26,1177


In [17]:
hittersmean['OBPOnTeam'] = (hittersmean['PA']*hittersmean['OBP']+2037.4)/(hittersmean['PA']+6100)
hittersmean['SLGOnTeam'] = (hittersmean['AB']*hittersmean['SLG']+2420)/(hittersmean['AB']+5500)
hittersmean['OPSValue'] = (hittersmean['OBPOnTeam']+hittersmean['SLGOnTeam']-.774)/.006

hittersmean['AVGValue'] = ((hittersmean['H']+1768)/(hittersmean['AB']+6100)-.267)/.0024
hittersmean['value'] = hittersmean['OPSValue']+(hittersmean['R']/8.92+hittersmean['HR']/7.5+hittersmean['RBI']/8.92+hittersmean['SB']/6.0)

In [18]:
#https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=1&season=2020&month=0&season1=2020&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=
positions = pd.read_csv("positionsold/FanGraphs Leaderboard.csv")

In [19]:
positions['Pos']=positions['Pos'].replace(['LF','CF','RF'],'OF')


In [20]:
positions_unique=positions.loc[(positions['G']>=10) | (positions['GS']>=5)].groupby('playerid')['Pos'].unique()


In [21]:
positions_unique.index=positions_unique.index.map(str)

In [22]:
hitterspos=pd.merge(hittersmean,positions_unique,how='left',left_on='playerid',right_index=True)


In [23]:
hitterspos

,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,InterSD,InterSK,IntraSD,playerid,OBPOnTeam,SLGOnTeam,OPSValue,AVGValue,value,Pos
0,Juan Soto,WSN,152,661,519,162,30,3,36,113,...,2.51,0.73,0.99,20123,0.345732,0.452675,4.067892,10.243680,35.028580,[OF]
1,Mike Trout,LAA,136,587,476,134,25,3,37,98,...,3.28,-1.31,0.74,10155,0.340671,0.450992,2.943903,9.263990,30.680675,[OF]
2,José Ramírez,CLE,148,633,551,149,35,4,35,100,...,3.15,0.63,0.84,13510,0.336256,0.448924,1.863361,8.844723,33.118219,[3B]
3,Mookie Betts,LAD,146,657,565,158,34,3,31,112,...,3.24,-0.33,0.66,13611,0.337792,0.446800,1.765427,9.155101,30.541511,"[2B, OF]"
4,Vladimir Guerrero Jr.,TOR,152,650,568,173,32,2,41,107,...,4.07,-0.06,1.44,19611,0.339296,0.453479,3.129255,10.038242,33.311169,[1B]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,Jose Rojas,LAA,28,95,87,18,3,0,3,11,...,5.24,1.32,1.51,19331,0.333049,0.438427,-0.420587,9.029080,2.612447,"[2B, 3B, OF]"
614,Magneuris Sierra,LAA,33,90,82,19,3,1,0,10,...,2.94,2.15,1.79,17023,0.333215,0.437914,-0.478521,9.193761,1.869760,[OF]
615,Albert Pujols,STL,45,168,156,37,6,0,7,17,...,2.48,-0.91,1.28,1177,0.332606,0.439145,-0.374795,8.967924,5.209510,[1B]
616,Miguel Cabrera,DET,111,461,415,104,16,0,13,45,...,2.94,-0.09,0.49,1744,0.332595,0.436422,-0.830577,8.473715,12.225626,[1B]


In [24]:
hitterspos['BestPos']=hitterspos['Pos'].apply(GetPosition)

hitterspos['rank']=hitterspos.groupby('BestPos')['value'].rank(ascending=False)
hitterspos['utilrank']=hitterspos['value'].rank(ascending=False)

###customized league replacement value
replacementvalue=hitterspos.groupby('BestPos').value.nlargest(10).groupby('BestPos').min().reset_index(name='PosValue')
replacementvalue['UtilValue']=float(hitterspos.loc[hitterspos['utilrank']==80]['value'])

replacementvalue['ReplacementValue']=pd.DataFrame([replacementvalue['UtilValue'], replacementvalue['PosValue']]).min()
#replacementvalue['ReplacementValue']=replacementvalue['UtilValue']
replacementvalue.loc[replacementvalue['BestPos'] == 'Util', 'ReplacementValue'] = replacementvalue['UtilValue']

In [25]:
replacementvalue

,BestPos,PosValue,UtilValue,ReplacementValue
0,1B,23.667162,21.721889,21.721889
1,2B,24.118792,21.721889,21.721889
2,3B,21.311333,21.721889,21.311333
3,C,14.252187,21.721889,14.252187
4,OF,27.490953,21.721889,21.721889
5,SS,21.869120,21.721889,21.721889
6,Util,12.347241,21.721889,21.721889


In [26]:
hittersfinal=hitterspos.reset_index().merge(replacementvalue,on='BestPos',how='left')

hittersfinal['VORP']=hittersfinal['value']-hittersfinal['ReplacementValue']


In [27]:
hittersfinal

,index,Name,Team,G,PA,AB,H,2B,3B,HR,...,AVGValue,value,Pos,BestPos,rank,utilrank,PosValue,UtilValue,ReplacementValue,VORP
0,0,Juan Soto,WSN,152,661,519,162,30,3,36,...,10.243680,35.028580,[OF],OF,1.0,1.0,27.490953,21.721889,21.721889,13.306691
1,1,Mike Trout,LAA,136,587,476,134,25,3,37,...,9.263990,30.680675,[OF],OF,3.0,8.0,27.490953,21.721889,21.721889,8.958786
2,2,José Ramírez,CLE,148,633,551,149,35,4,35,...,8.844723,33.118219,[3B],3B,1.0,3.0,21.311333,21.721889,21.311333,11.806885
3,3,Mookie Betts,LAD,146,657,565,158,34,3,31,...,9.155101,30.541511,"[2B, OF]",2B,2.0,9.0,24.118792,21.721889,21.721889,8.819622
4,4,Vladimir Guerrero Jr.,TOR,152,650,568,173,32,2,41,...,10.038242,33.311169,[1B],1B,1.0,2.0,23.667162,21.721889,21.721889,11.589280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,613,Jose Rojas,LAA,28,95,87,18,3,0,3,...,9.029080,2.612447,"[2B, 3B, OF]",2B,97.0,456.0,24.118792,21.721889,21.721889,-19.109442
614,614,Magneuris Sierra,LAA,33,90,82,19,3,1,0,...,9.193761,1.869760,[OF],OF,153.0,499.0,27.490953,21.721889,21.721889,-19.852129
615,615,Albert Pujols,STL,45,168,156,37,6,0,7,...,8.967924,5.209510,[1B],1B,50.0,375.0,23.667162,21.721889,21.721889,-16.512378
616,616,Miguel Cabrera,DET,111,461,415,104,16,0,13,...,8.473715,12.225626,[1B],1B,40.0,234.0,23.667162,21.721889,21.721889,-9.496263


In [28]:

########################
#CHANGE THE COLUMNS BELOW BASED ON SGP VALUES FOR YOUR LEAGUE
pitchersmean['ERA']=((525.55+pitchersmean['ER'])*9/(1350.0+pitchersmean['IP'])-3.5)/-0.1062
pitchersmean['WHIP'] =((1620.0+pitchersmean['H']+pitchersmean['BB'])/(1350.0+pitchersmean['IP'])-1.2)/-0.0201
pitchersmean['value'] = pitchersmean['ERA']+pitchersmean['WHIP']+(pitchersmean['W']/4.05+pitchersmean['K/9']*pitchersmean['IP']/(55.0*9.0)+(pitchersmean['SV']+pitchersmean['HLD'])/10.0)
########################

pitchersmean['utilrank']=pitchersmean['value'].rank(ascending=False)

###customized league replacement value
pitchersmean['ReplacementValue']=float(pitchersmean.loc[pitchersmean['utilrank']==100]['value'])

pitchersmean['VORP']=pitchersmean['value']-pitchersmean['ReplacementValue']


In [29]:
pitchersmean

,Name,Team,GS,G,IP,W,L,QS,SV,HLD,...,ADP,-1.2,InterSD,InterSK,IntraSD,playerid,value,utilrank,ReplacementValue,VORP
0,Jacob deGrom,NYM,25,25,149.0,11,5,16,0,0,...,24.0,NaN,6.72,-0.05,1.97,10954,8.607155,2.0,4.009148,4.598007
1,Corbin Burnes,MIL,28,29,169.0,12,6,16,0,0,...,10.4,NaN,2.91,0.07,1.35,19361,8.125532,3.0,4.009148,4.116384
2,Gerrit Cole,NYY,30,30,189.0,15,8,18,0,0,...,6.9,NaN,4.26,0.27,1.57,13125,9.659825,1.0,4.009148,5.650677
3,Zack Wheeler,PHI,28,29,180.0,12,9,16,0,0,...,34.6,NaN,4.20,0.15,0.85,10310,6.671796,14.0,4.009148,2.662647
4,Shane Bieber,CLE,28,28,173.0,13,8,17,0,0,...,31.5,NaN,7.94,-1.19,1.08,19427,7.829967,5.0,4.009148,3.820819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,Chi Chi González,NaN,8,42,70.0,2,3,1,0,2,...,999.0,NaN,8.66,-0.77,0.68,14663,-0.066485,790.0,4.009148,-4.075633
807,Anthony Gose,CLE,0,44,46.0,3,3,0,0,6,...,601.0,NaN,4.31,-0.66,0.52,5097,1.288916,388.0,4.009148,-2.720233
808,Josh Tomlin,NaN,0,42,45.0,2,2,0,0,3,...,999.0,NaN,0.94,-0.66,0.65,9388,0.635514,521.0,4.009148,-3.373635
809,Jhoulys Chacín,COL,1,53,62.0,3,3,1,0,11,...,999.0,NaN,1.75,-0.24,0.51,2608,1.350655,381.0,4.009148,-2.658493


In [30]:
final=pd.concat([hittersfinal,pitchersmean])


In [31]:
final['VORP']=final['VORP'].round(2)

In [32]:
final.sort_values('VORP', ascending=False)

,index,Name,Team,G,PA,AB,H,2B,3B,HR,...,QS,SV,HLD,ER,WHIP,K/9,BB/9,ERA,FIP,RA9-WAR
0,0.0,Juan Soto,WSN,152,661.0,519.0,162,30.0,3.0,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,José Ramírez,CLE,148,633.0,551.0,149,35.0,4.0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,Vladimir Guerrero Jr.,TOR,152,650.0,568.0,173,32.0,2.0,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7.0,Bryce Harper,PHI,150,646.0,528.0,146,32.0,2.0,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,33.0,Shohei Ohtani,LAA,143,603.0,518.0,135,25.0,5.0,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,475.0,Ali Sánchez,NaN,11,37.0,35.0,8,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
521,521.0,Greg Deichmann,NaN,9,28.0,25.0,5,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
567,567.0,Taylor Davis,NaN,9,29.0,26.0,6,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557,557.0,Nick Martini,NaN,8,20.0,17.0,4,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
final.sort_values('VORP', ascending=False).nlargest(200,['VORP']).to_csv("ATC.csv")

In [34]:
final[['Name','Team','Pos','BestPos','G','GS','W','SV','VORP','ADP','rank','utilrank']].sort_values('VORP', ascending=False).nlargest(200,['VORP']).to_csv("ATC.csv")